# make sure python version and updated requirements still  behave

first in terminal in the root directory this notebook is in (eg /Users/snyd535/Documents/GitHub/stitches,
automatically there via terminal in pycharm), do
`python setup.py install`

In [1]:
import pandas as pd
import pkg_resources
import stitches as stitches
import xarray as xr

In [2]:
x = stitches.fx_pangeo.fetch_pangeo_table()
x


# would probably be good to test installing package and running this
# jupyter notebook directly from command line in a fresh clone, no
# pycharm at all

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year,version
0,HighResMIP,CMCC,CMCC-CM2-HR4,highresSST-present,r1i1p1f1,Amon,ps,gn,gs://cmip6/CMIP6/HighResMIP/CMCC/CMCC-CM2-HR4/...,NaN,20170706
1,HighResMIP,CMCC,CMCC-CM2-HR4,highresSST-present,r1i1p1f1,Amon,rsds,gn,gs://cmip6/CMIP6/HighResMIP/CMCC/CMCC-CM2-HR4/...,NaN,20170706
2,HighResMIP,CMCC,CMCC-CM2-HR4,highresSST-present,r1i1p1f1,Amon,rlus,gn,gs://cmip6/CMIP6/HighResMIP/CMCC/CMCC-CM2-HR4/...,NaN,20170706
3,HighResMIP,CMCC,CMCC-CM2-HR4,highresSST-present,r1i1p1f1,Amon,rlds,gn,gs://cmip6/CMIP6/HighResMIP/CMCC/CMCC-CM2-HR4/...,NaN,20170706
4,HighResMIP,CMCC,CMCC-CM2-HR4,highresSST-present,r1i1p1f1,Amon,psl,gn,gs://cmip6/CMIP6/HighResMIP/CMCC/CMCC-CM2-HR4/...,NaN,20170706
...,...,...,...,...,...,...,...,...,...,...,...
514813,CMIP,EC-Earth-Consortium,EC-Earth3-Veg,historical,r1i1p1f1,Amon,tas,gr,gs://cmip6/CMIP6/CMIP/EC-Earth-Consortium/EC-E...,NaN,20211207
514814,CMIP,EC-Earth-Consortium,EC-Earth3-Veg,historical,r1i1p1f1,Amon,tauu,gr,gs://cmip6/CMIP6/CMIP/EC-Earth-Consortium/EC-E...,NaN,20211207
514815,CMIP,EC-Earth-Consortium,EC-Earth3-Veg,historical,r1i1p1f1,Amon,hur,gr,gs://cmip6/CMIP6/CMIP/EC-Earth-Consortium/EC-E...,NaN,20211207
514816,CMIP,EC-Earth-Consortium,EC-Earth3-Veg,historical,r1i1p1f1,Amon,hus,gr,gs://cmip6/CMIP6/CMIP/EC-Earth-Consortium/EC-E...,NaN,20211207


In [21]:
pangeo_path = pkg_resources.resource_filename('stitches', 'data/pangeo_table.csv')
pangeo_data = pd.read_csv(pangeo_path)
pangeo_data = pangeo_data[((pangeo_data['variable'] == 'tas') | (pangeo_data['variable'] == 'pr') | (pangeo_data['variable'] == 'psl'))
                          & ((pangeo_data['domain'] == 'Amon') ) ].copy()


outdir = '/Users/snyd535/Documents/task11a-topdown-clim-ML/Tebaldi_etal_2022_ESD/paper_data/GRIDDED/comparison'
pangeo_data.head()

,model,experiment,ensemble,variable,zstore,domain
44,GISS-E2-1-G,1pctCO2,r1i1p1f1,tas,gs://cmip6/CMIP6/CMIP/NASA-GISS/GISS-E2-1-G/1p...,Amon
56,GISS-E2-1-G,1pctCO2,r1i1p1f1,pr,gs://cmip6/CMIP6/CMIP/NASA-GISS/GISS-E2-1-G/1p...,Amon
71,GISS-E2-1-G,1pctCO2,r1i1p1f1,psl,gs://cmip6/CMIP6/CMIP/NASA-GISS/GISS-E2-1-G/1p...,Amon
138,GISS-E2-1-G,abrupt-4xCO2,r1i1p1f1,psl,gs://cmip6/CMIP6/CMIP/NASA-GISS/GISS-E2-1-G/ab...,Amon
148,GISS-E2-1-G,abrupt-4xCO2,r1i1p1f1,tas,gs://cmip6/CMIP6/CMIP/NASA-GISS/GISS-E2-1-G/ab...,Amon


In [30]:
esm = 'MIROC6'
exp = ['ssp245', 'ssp370']
ens = ['r1i1p1f1',
       'r2i1p1f1',
       'r3i1p1f1']

data_list = pangeo_data[(pangeo_data['model'] == esm) &
(pangeo_data['variable'] == 'psl') &
(pangeo_data['experiment'].isin(exp)) &
(pangeo_data['ensemble'].isin(ens))].copy()

for i  in range(0, len(data_list)):
    data = data_list.iloc[i].copy()
    ds = stitches.fx_pangeo.fetch_nc(data.zstore)
    ds.to_netcdf(outdir + '/' + data.model + '_psl_'  + data.experiment + '_' + data.ensemble + '.nc')

    del(data)